In [12]:
import MDAnalysis as mda
import numpy as np
import sys
nav = mda.units.N_Avogadro
import math
import os
if not os.path.isfile('./topol.top'):
    print("File 'topol.top' is missing")
if not os.path.isfile('./make_tpr.mdp'):
    print("File 'make_tpr.mdp' is missing")
if not os.path.isdir('./toppar'):
    print("Directory 'toppar' is missing")

In [28]:
class PutAtomsIntoGrid(object):
    def __init__(self,molecule,membrane,Z_box,Z_min,Z_scale,N,output):
        self.molecule = molecule
        self.membrane = membrane
        self.Z_box = Z_box
        self.Z_min = Z_min
        self.Z_scale = Z_scale
        self.N = N
        self.charge=1
        self.output = output
        if os.path.isfile('./topol.top') and os.path.isfile('./make_tpr.mdp') and os.path.isdir('./toppar'):
            self.prepare_box()
            self.solvate()
        else:
            print("Some of the required files are missing:")
        if not os.path.isfile('./topol.top'):
            print("\t File 'topol.top' is missing")
        if not os.path.isfile('./make_tpr.mdp'):
            print("\t File 'make_tpr.mdp' is missing")
        if not os.path.isdir('./toppar'):
            print("\t Directory 'toppar' is missing")


    #read the atomic names, and number of electrons;
    #make the dictonary of atom name and number of electrons for POPC molecule  and water
    def prepare_box(self):
        count=len(open(self.membrane).readlines(  ))
        with open (self.membrane) as f:
            rawdata = f.read().split('\n')
            membrane_coor = rawdata[2:count-1]
            membrane_atoms= int(rawdata[1:2][0][:])
            pre_all_data = np.asarray([l.split() for l in rawdata])
            box_size = rawdata[count-1:count]
            box_size = np.asarray([l.split() for l in box_size],dtype=float)
        box_size[0,2]= (self.Z_box+self.Z_min)/10
        X_box = box_size[0,0]*10 
        Y_box = box_size[0,1]*10
        
    
       
                
                
        
        with open (self.molecule) as f:
            rawdata = f.read().split('\n')
            pre_all_data = np.asarray([l.split() for l in rawdata])
            
      
        #number of atoms per molecule     
        atoms=pre_all_data.shape[0]-1

        #read the coordinates from the molecule
        all_data=np.zeros((atoms,12)).astype(object)
        for i in range (0,atoms):
            for j in (0,2,3,4,11):
                all_data[i,j]=pre_all_data[i][j]
            for j in (6,7,8,9,10):
                all_data[i,j]=float(pre_all_data[i][j])
            for j in (1,5):
                all_data[i,j]=int(pre_all_data[i][j])

        membrane_atoms+=self.N*atoms
        
        with open("membrane_stack.gro","w") as f:
            f.write("POPC/EDI \n{} \n".format(membrane_atoms))
            for row in membrane_coor:
                f.write("{}\n".format(row))
            
            
        gro_data=np.zeros((atoms,7)).astype(object)
        gro_data[:,0]=all_data[:,5]
        gro_data[:,1]=all_data[:,3]
        gro_data[:,2]=all_data[:,2]
        gro_data[:,3]=all_data[:,1]
        gro_data[:,4]=all_data[:,6]
        gro_data[:,5]=all_data[:,7]
        gro_data[:,6]=all_data[:,8]


        #read the coordinates from the molecule
        init_coord=np.copy(all_data[:,6:9])
        #measure the size of the molecule
        X_size=np.max(init_coord[:,0])-np.min(init_coord[:,0])
        Y_size=np.max(init_coord[:,1])-np.min(init_coord[:,1])
        Z_size=np.max(init_coord[:,2])-np.min(init_coord[:,2])
        #put the molecule to the ground state, e.g. x=0,y=0, z=Z_min
        min_coord=np.copy(init_coord)
        min_coord[:,0]=init_coord[:,0]-np.min(init_coord[:,0])
        min_coord[:,1]=init_coord[:,1]-np.min(init_coord[:,1])
        min_coord[:,2]=init_coord[:,2]-np.min(init_coord[:,2])+self.Z_min
        #estimated maximum number of molecules
        print("The maximum number of molecules in the box is: %5.1f" % (round(X_box/X_size/2)*round(Y_box/Y_size/2)*round(self.Z_box/Z_size/2)))


        #calculate the number of molecules in x,y,z directions, check if it is suffitient, if not round all directions up
        #check for the ratios betweent the maximal numbers of molecules per dimension (count 2 times less for Z direction)
        a=X_box/X_size/Y_box*Y_size
        b=Y_box/Y_size/self.Z_box*Z_size*self.Z_scale
        amount_X=math.floor((a*a*b*self.N)**(1/3))
        amount_Y=math.floor((b*self.N/a)**(1/3))
        amount_Z=math.floor((self.N/a/b/b)**(1/3))
        if amount_X==0:
            amount_X=1
        if amount_Y==0:
            amount_Y=1
        if amount_Z==0:
            amount_Z=1
            
        print("Box prepared for %5.1f molecules" % (amount_X*amount_Y*amount_Z))
        print("\t X-direction: %5.1f molecules \n \t Y-direction: %5.1f molecules \n \t Z-direction: %5.1f molecules \n" % (amount_X,amount_Y,amount_Z))
        if amount_X*amount_Y*amount_Z<self.N and (amount_X<=amount_Y):
            print("Less then required, increasing box size... \n")
            amount_X=math.ceil((a*a*b*self.N)**(1/3))
            amount_Y=math.floor((b*self.N/a)**(1/3))
            amount_Z=math.floor((self.N/a/b/b)**(1/3))
            print("Box now prepared for %5.1f molecules" % (amount_X*amount_Y*amount_Z))
            print("\t X-direction: %5.1f molecules \n \t Y-direction: %5.1f molecules \n \t Z-direction: %5.1f molecules" % (amount_X,amount_Y,amount_Z))
        if amount_X*amount_Y*amount_Z<self.N and (amount_X>amount_Y):
            print("Less then required, increasing box size... \n")
            amount_X=math.floor((a*a*b*self.N)**(1/3))
            amount_Y=math.ceil((b*self.N/a)**(1/3))
            amount_Z=math.floor((self.N/a/b/b)**(1/3))
            print("Box now prepared for %5.1f molecules" % (amount_X*amount_Y*amount_Z))
            print("\t X-direction: %5.1f molecules \n \t Y-direction: %5.1f molecules \n \t Z-direction: %5.1f molecules" % (amount_X,amount_Y,amount_Z))    
        if amount_X*amount_Y*amount_Z<self.N:
            print("Less then required, increasing box size... \n")
            amount_X=math.ceil((a*a*b*self.N)**(1/3))
            amount_Y=math.ceil((b*self.N/a)**(1/3))
            amount_Z=math.floor((self.N/a/b/b)**(1/3))
            print("Box now prepared for %5.1f molecules" % (amount_X*amount_Y*amount_Z))
            print("\t X-direction: %5.1f molecules \n \t Y-direction: %5.1f molecules \n \t Z-direction: %5.1f molecules" % (amount_X,amount_Y,amount_Z))    
        if amount_X*amount_Y*amount_Z<self.N:
            print("Less then required, increasing box size... \n")
            amount_X=int(math.ceil((a*a*b*self.N)**(1/3)))
            amount_Y=int(math.ceil((b*self.N/a)**(1/3)))
            amount_Z=int(math.ceil((self.N/a/b/b)**(1/3)))
            print("Box now prepared for %5.1f molecules" % (amount_X*amount_Y*amount_Z))
            print("\t X-direction: %5.1f molecules \n \t Y-direction: %5.1f molecules \n \t Z-direction: %5.1f molecules" % (amount_X,amount_Y,amount_Z))

    
        count=0
        coord=np.copy(min_coord)
        X_separ=X_box/amount_X
        Y_separ=Y_box/amount_Y
        Z_separ=self.Z_box/amount_Z

       
        
        amount_Z=int(amount_Z)
        amount_Y=int(amount_Y)
        amount_X=int(amount_X)

        for i in range (0,amount_Z):
            for j in range (0,amount_Y):
                for k in range (0,amount_X):
                    if count<self.N:
                        coord[:,0]=min_coord[:,0]+k*X_separ
                        coord[:,1]=min_coord[:,1]+j*Y_separ
                        coord[:,2]=min_coord[:,2]+i*Z_separ
                
                        count+=1
                        gro_data[:,4:7]=coord/10
                        gro_data[:,0]=count
                        with open("membrane_stack.gro","a") as f:
                            for row in gro_data:
                                f.write("{:>5}{:3}{:>7}{:>5}{: >8.3f}{: >8.3f}{: >8.3f}\n".format(*row))
                                
                        gro_data[:,3]+=atoms
        with open("membrane_stack.gro","a") as f:
            f.write("{: >10.5f}".format(box_size[0,0]))
            f.write("{: >10.5f}".format(box_size[0,1]))
            f.write("{: >10.5f}".format(box_size[0,2]))
            
    def solvate(self):
        !gmx solvate -cp membrane_stack.gro -o solvated.gro -p topol -quiet
        !sed 's/SOL/TIP3/g' topol.top > {self.output}.top
        if self.charge != 0:
            charge=self.charge*self.N+4
            print(charge)
            !gmx grompp -f make_tpr.mdp -p {self.output} -c solvated -o solvated -maxwarn 2
            if charge > 0:
                print(charge)
                !echo "TIP3" |gmx genion -s solvated.tpr -nn {charge} -nname CLA -o {self.output} -p {self.output}
            elif charge < 0:
                print(charge)
                !echo "TIP3" |gmx genion -s solvated.tpr -np {charge} -pname SOD -o {self.output} -p {self.output}
                

In [29]:
PutAtomsIntoGrid("../0-dibucaine/topology/1-dibucaine.pdb","0-membrane.gro",15,65,1.4,4,"52mM")

The maximum number of molecules in the box is:  12.0
Box prepared for   4.0 molecules
	 X-direction:   2.0 molecules 
 	 Y-direction:   2.0 molecules 
 	 Z-direction:   1.0 molecules 

Reading solute configuration
POPC/EDI
Containing 27240 atoms in 208 residues
Reading solvent configuration
216H2O,WATJP01,SPC216,SPC-MODEL,300K,BOX(M)=1.86206NM,WFVG,MAR. 1984
Containing 648 atoms in 216 residues

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys.

8
                      :-) GROMACS - gmx genion, 2018.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS 

In [23]:
pwd

'/home/local/nenciric/Documents/simulations/prepare/dibucaine/26mM'